In [86]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import train_reg
import config
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# prepare data

In [196]:
dsets = ['clath_aux+gak_a7d2_new'] # this is one of the large datasets
#feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
              #['mean_total_displacement', 'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
df_full = None
for dset in dsets:
    df = data.get_data(dset=dset)
    df = df[df.lifetime > 15] # only keep hard tracks
    df = df[df.cell_num.isin(config.DSETS[dset]['train'])] # exclude held-out test data
    feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
    
    # downsample tracks
    length = 40
    df['X_same_length'] = [features.downsample(df.iloc[i]['X'], length)
                           for i in range(len(df))] # downsampling
    # normalize tracks
    df = features.normalize_track(df, track='X_same_length')
    
    # regression response
    df = train_reg.add_sig_mean(df)     
    
    # remove extraneous feats
    df = df[feat_names + meta]
    df = df.dropna() 
    
    # normalize features
    for feat in feat_names:
        if 'X_same_length' not in feat:
            df = features.normalize_feature(df, feat)
    
    # merge all dsets
#     if df_full is None:
#         df_full = deepcopy(df)
#     else:
#         df_full = df_full.merge(df)

../src/train_reg.py:83: RuntimeWarning: invalid value encountered in less
  sigs = np.array(r[f'{track}_pvals']) < 0.05


## train neural net

In [198]:
# decide on architecture
#out_dir = f'{DIR_RESULTS}/dec10_deep'
out_dir = f'results/regression/deep_learning/Dec11'

In [ ]:
os.makedirs(out_dir, exist_ok=True)
outcome_def = 'Y_sig_mean_normalized'
num_epochs = 200
num_hidden = 40
for model_type in ['nn_cnn', 'fcnn']: # = 'nn_cnn' # 'nn_lstm', 'fcnn', 'nn_cnn', 'nn_attention'
    train_reg.train_reg(df,
                        feat_names=feat_names,
                        track_name='X_same_length_normalized',
                        model_type=model_type, 
                        outcome_def=outcome_def,
                        out_name=oj(out_dir, f'{dset}_{outcome_def}_{model_type}_{num_epochs}_{num_hidden}.pkl'),
                        fcnn_hidden_neurons=num_hidden,
                        fcnn_epochs=num_epochs)

0it [00:00, ?it/s]

Looping over cv...
fitting dnn...


1it [05:12, 312.85s/it]

0.35479134591952133
fitting dnn...


2it [10:06, 307.01s/it]

...

0.43377543675050234
fitting dnn...


4it [20:22, 307.06s/it]

0.3394764714616395
fitting dnn...


5it [25:32, 307.85s/it]

0.3004298821318502
fitting dnn...


# analyze results

In [204]:
results = train_reg.load_results(out_dir)
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['r2'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r

{'r2': [0.35292968297812044, 0.41321238194309773, 0.4213124728107559, 0.34441119955718824, 0.3079348229670724, 0.33056927360656363, 0.40911555853457926, 0.3668184848391457], 'pearsonr': [0.5966738886061014, 0.6432680373493718, 0.6535560424380565, 0.587823062433223, 0.5581190719310201, 0.5772658447895979, 0.6398673829480921, 0.6081436611725487]}
dict_keys(['r2', 'pearsonr'])
{'r2': [0.3657887428322416, 0.4122538569014256, 0.4263862191200558, 0.36088525550063644, 0.3197916085025576, 0.34020802664287686, 0.4069579726894177, 0.3861140929989211], 'pearsonr': [0.6048387155814439, 0.6427490157692443, 0.6548975467904691, 0.6019669761449945, 0.5661079070233239, 0.5850302582256015, 0.6383533969995763, 0.6234599944148711]}
dict_keys(['r2', 'pearsonr'])
{'r2': [0.3453869748471894, 0.4074891206827066, 0.4197802515063189, 0.32252513243006153, 0.2724102411905601, 0.32948402443986513, 0.4097043441717406, 0.368706918317809], 'pearsonr': [0.589113214445168, 0.6405624654427993, 0.6508771489214968, 0.5684

,cv_accuracy_by_cell,pearsonr,r2
model_type,,,
clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_lstm,"[0.3657887428322416, 0.4122538569014256, 0.426...",0.614,0.377
clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_cnn,"[0.35292968297812044, 0.41321238194309773, 0.4...",0.607,0.368
clath_aux+gak_a7d2_new_Y_sig_mean_normalized_fcnn,"[0.3453869748471894, 0.4074891206827066, 0.419...",0.602,0.358
